In [1]:
import numpy as np
from Music_Style_Transfer_master.project.midi_handler import score2midi, midi2score
from Music_Style_Transfer_master.project.utils import add_beat, padding_v2, load_model, get_representation, dataset_import
from Music_Style_Transfer_master.project.test import style_transfer
from Music_Style_Transfer_master.project.model_transformer_relative import _2way_transformer_wavenet_absolute_pitch, _2way_transformer_wavenet_absolute_pitch_new_embedding

import tqdm
import pypianoroll as pr
import tensorflow as tf
import os

import matplotlib.pyplot as plt
#from keras.models import load_model
%matplotlib inline

# 33:97 A1~C7  21:109 A0~C8
# 7 octave 21:105  A0~G#7  
def midi2score(path, beat_resolution, pitch_range=np.arange(21, 109)):
    tmp = pr.Multitrack(filename=path, beat_resolution=beat_resolution)   
    tmp.binarize()
    tmp.assign_constant(1)
    # TODO: use variable to handle pitch range
    score = tmp.get_merged_pianoroll()[:, pitch_range]
    
    return score

def load_model(path_model):
    model = _2way_transformer_wavenet_absolute_pitch_new_embedding(len_context = 8*4*beat_resolution,
                                                 n_octave = 7,
                                                 size_embedding = 86,
                                                 n_transf_layers = 4,
                                                 n_transf_downsample = 0,
                                                 n_conv_layers = 7,
                                                 context_layers = 1)
    model.load_weights(path_model)
    
    return model



Using TensorFlow backend.


In [15]:
beat_resolution = 8
pitch_range = 84
len_context = beat_resolution*4*8
max_len = beat_resolution*4*4

path_model = "../../../scratch/wtl272/projects/Music_style_transfer/runs/run_67/model/model.hdf5"
model = load_model(path_model)

In [16]:
path = "output_midi/Results_transformer/RockyRaccoon_bach_model67.mid"
score = midi2score(path, beat_resolution, pitch_range=np.arange(21, 105))
score, score_meta = get_representation(score, pitch_range, beat_resolution, len_context)
score = (np.array(score[:, :, 2]) > 0).astype(int)

In [17]:
score_melody = midi2score('input_midi/songs_general/RockyRaccoon_main.mid', beat_resolution, pitch_range=np.arange(21, 105))

In [18]:
onset_map = []
offset_map = []
for t in range(len_context, len(score) - len_context):

    scaler = np.arange(1, pitch_range + 1)

    outputs = np.zeros(shape=(pitch_range, 1))
    left_features = (np.array(score[t-len_context:t]) > 0).astype(int)*scaler
    left_metas = np.array(score_meta[t-len_context:t])
    right_features = (np.array(score[t+1:t+1+len_context]) > 0).astype(int)*scaler
    right_metas = np.array(score_meta[t+1:t+1+len_context]) 
    central_metas = np.array(score_meta[t])
    central_metas = np.reshape(central_metas, (1, -1))

    i = 0
    central_features = (np.array(score[t, :]) > 0).astype(int)
    central_features[i:] = 2
    central_features = np.reshape(central_features, (1, -1))
    
    p = model.predict([left_features[None, :, :], left_metas[None, :, :],
                               central_features[None, :, :], central_metas[None, :, :],
                               right_features[None, :, :], right_metas[None, :, :]]
                             )
    onset_map.append(p[1][0][-1][84:84*2]*p[2][0][0][84:84*2])
    offset_map.append(p[1][0][-1][84*2:]*p[2][0][0][84*2:])

In [19]:
# model 68  threshold - >0.05

threshold = 0.05
o_map = (np.array(onset_map)>threshold).astype(int)
score[len_context-1:-(len_context+1)] -=o_map
score = (score>0).astype(int)

In [20]:
score2midi("output_midi/Results_onset_fix/RockyRaccoon_bach_model67.mid", 
               score[len_context:-(len_context)] , beat_resolution, 120, 
               pitch_range=np.arange(21, 105), melody_constraint=True, melody=score_melody)